In [8]:
import fitz  # PyMuPDF

def extract_text_with_ids(pdf_path):
    doc = fitz.open(pdf_path)
    text_with_ids = []
    word_id = 1  # Start the counter from 1

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        word_list = page.get_text("words")  # list of words on page

        for word in word_list:
            text_with_ids.append({
                "id": word_id,
                "text": word[4],
                "position": word[:4],
                "page": page_num
            })
            word_id += 1  # Increment the counter for each word

    return text_with_ids

# Path to your PDF file
pdf_path = "Eren CV Muster.docx.pdf"

# Extract text with IDs
pdf_text_with_ids = extract_text_with_ids(pdf_path)

# Print the IDs and the corresponding text
for item in pdf_text_with_ids:
    print(f"ID: {item['id']}, Text: {item['text']}")

ID: 1, Text: Eren
ID: 2, Text: Kalinsazlioglu
ID: 3, Text: eren@enpoi.com
ID: 4, Text: |
ID: 5, Text: +49
ID: 6, Text: 17659532268
ID: 7, Text: |
ID: 8, Text: Niedenstein,
ID: 9, Text: Germany
ID: 10, Text: |
ID: 11, Text: Website|
ID: 12, Text: YouTube|Github
ID: 13, Text: About
ID: 14, Text: me
ID: 15, Text: I
ID: 16, Text: have
ID: 17, Text: passion
ID: 18, Text: and
ID: 19, Text: an
ID: 20, Text: aptitude
ID: 21, Text: for
ID: 22, Text: innovation,
ID: 23, Text: creative
ID: 24, Text: problem-solving,
ID: 25, Text: and
ID: 26, Text: analytical
ID: 27, Text: thinking.
ID: 28, Text: Theoretical
ID: 29, Text: and
ID: 30, Text: practical
ID: 31, Text: knowledge
ID: 32, Text: of
ID: 33, Text: electronics,
ID: 34, Text: mechanics,
ID: 35, Text: and
ID: 36, Text: software
ID: 37, Text: development
ID: 38, Text: processes
ID: 39, Text: are
ID: 40, Text: my
ID: 41, Text: strengths.
ID: 42, Text: I
ID: 43, Text: started
ID: 44, Text: making
ID: 45, Text: paper
ID: 46, Text: models
ID: 47, Te

In [12]:
def replace_word_in_pdf(original_pdf_path, output_pdf_path, word_id, new_text):
    doc = fitz.open(original_pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        word_list = page.get_text("words")

        for word in word_list:
            current_id = "_".join(map(str, word[:4]))
            if current_id == word_id:
                rect = fitz.Rect(word[:4])  # position of the word to be replaced
                page.add_redact_annot(rect)  # prepare to redact the area
                page.apply_redactions()  # apply redaction to remove old text
                page.insert_text(rect.tl, new_text, fontsize=11)  # insert new text

    doc.save(output_pdf_path)

replace_word_in_pdf("/home/adam/JobJourneyGuide/Eren CV Muster.docx.pdf", "/home/adam/JobJourneyGuide/updated.pdf", "5", "Lorem")